## Init

In [ ]:
#setup for serving a standalone jupyter notebook
# !pip install voila
# !jupyter serverextension enable voila —sys-prefix

In [4]:
from utils import *
from fastai.vision.widgets import *

### Bing API Key

In [5]:
key = '7d5823f527b240ccb1a74fb1ca2d1400'

## Download and clean bus and skoolie images from Bing

In [6]:
bus_types = 'skoolie exterior', 'school bus'
path = Path('buses')

In [7]:
if not path.exists():
    path.mkdir()
    for o in bus_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o}')
        download_images(dest, urls=results.attrgot('content_url'))

In [8]:
fns = get_image_files(path)
failed = verify_images(fns)
failed.map(Path.unlink);

(#304) [Path('buses/skoolie exterior/00000003.jpg'),Path('buses/skoolie exterior/00000006.jpg'),Path('buses/skoolie exterior/00000002.jpg'),Path('buses/skoolie exterior/00000007.jpg'),Path('buses/skoolie exterior/00000001.jpg'),Path('buses/skoolie exterior/00000000.jpg'),Path('buses/skoolie exterior/00000008.jpg'),Path('buses/skoolie exterior/00000005.jpeg'),Path('buses/skoolie exterior/00000009.jpg'),Path('buses/skoolie exterior/00000004.jpg')...]

## Build the model

In [11]:
buses = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [14]:
buses = buses.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = buses.dataloaders(path)

## Train the Model

In [15]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(8)

epoch,train_loss,valid_loss,error_rate,time
0,1.175244,2.885865,0.500000,00:07


epoch,train_loss,valid_loss,error_rate,time
0,0.572239,1.755867,0.416667,00:06
1,0.408407,0.806308,0.233333,00:06
2,0.318753,0.486078,0.200000,00:06
3,0.275686,0.341037,0.100000,00:07
4,0.232505,0.277841,0.100000,00:06
5,0.196155,0.246650,0.083333,00:06
6,0.169485,0.227768,0.083333,00:07
7,0.147452,0.217673,0.050000,00:06


## Build a Notebook app for uploading and classifying a new image

In [20]:
btn_upload = widgets.FileUpload()

btn_run = widgets.Button(description='Classify')

out_pl = widgets.Output()

lbl_pred = widgets.Label()

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
btn_run.on_click(on_click_classify)
    

In [21]:
VBox([widgets.Label('Select your bus!'), 
      btn_upload, btn_run, out_pl, lbl_pred])